In [1]:
!pip install gensim

In [24]:
pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.0
    Uninstalling transformers-4.35.0:
      Successfully uninstalled transformers-4.35.0


In [2]:
from google.colab import files
files.upload()  # upload the kaggle.json you downloaded


Saving kaggle.json to kaggle (3).json


{'kaggle (3).json': b'{"username":"navyagn","key":"3f7f2f2cc0750574da55f7d5e1632479"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d vangap/indian-supreme-court-judgments


Dataset URL: https://www.kaggle.com/datasets/vangap/indian-supreme-court-judgments
License(s): CC-BY-NC-SA-4.0
indian-supreme-court-judgments.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!unzip indian-supreme-court-judgments.zip -d /content/indian_judgments

Archive:  indian-supreme-court-judgments.zip
replace /content/indian_judgments/judgments.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/indian_judgments/judgments.csv  
replace /content/indian_judgments/pdfs/-0___jonew__judis__10166.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [6]:
import pandas as pd
import os
import shutil

base_dir = '/content/indian_judgments'  # location of extracted dataset
csv_path = os.path.join(base_dir, 'judgments.csv')
pdf_dir = os.path.join(base_dir, 'pdfs')

# ======== STEP 1: Load dataset ========
judgments_df = pd.read_csv(csv_path)

# Keep only English judgments (where language is NaN)
judgments_df = judgments_df[judgments_df['language'].isna()].reset_index(drop=True)

# ======== STEP 2: Get list of PDFs ========
pdf_files = os.listdir(pdf_dir)
print("Total PDFs available:", len(pdf_files))

# ======== STEP 3: Filter judgments that have matching PDF ========
judgments_df['pdf_exists'] = judgments_df['diary_no'].astype(str).apply(
    lambda x: any(x in fname for fname in pdf_files)
)
judgments_df = judgments_df[judgments_df['pdf_exists']].reset_index(drop=True)
print("Judgments with PDFs:", len(judgments_df))

# ======== STEP 4: Take 2000 random samples ========
reduced_df = judgments_df.sample(n=2000, random_state=42).reset_index(drop=True)

# ======== STEP 5: Add full PDF path column ========
reduced_df['pdf_path'] = reduced_df['diary_no'].astype(str).apply(
    lambda x: next((os.path.join(pdf_dir, f) for f in pdf_files if x in f), None)
)

# ======== STEP 6: Save reduced CSV ========
reduced_csv_path = '/content/reduced_judgments.csv'
reduced_df.to_csv(reduced_csv_path, index=False)
print("Reduced CSV saved at:", reduced_csv_path)

# ======== STEP 7: Copy corresponding PDFs to a new folder ========
reduced_pdf_dir = '/content/reduced_pdfs'
os.makedirs(reduced_pdf_dir, exist_ok=True)

for pdf_path in reduced_df['pdf_path']:
    if pdf_path and os.path.exists(pdf_path):
        shutil.copy(pdf_path, reduced_pdf_dir)

print("Reduced PDFs copied to:", reduced_pdf_dir)
print("Number of PDFs in reduced folder:", len(os.listdir(reduced_pdf_dir)))


Total PDFs available: 48294
Judgments with PDFs: 46212
Reduced CSV saved at: /content/reduced_judgments.csv
Reduced PDFs copied to: /content/reduced_pdfs
Number of PDFs in reduced folder: 1944


In [7]:
reduced_csv_path = '/content/reduced_judgments.csv'
reduced_pdf_dir = '/content/reduced_pdfs'

# Load CSV
df = pd.read_csv(reduced_csv_path)
# Get list of PDFs in the folder
pdf_files = os.listdir(reduced_pdf_dir)

# Function to check if PDF exists for each diary_no
def check_pdf_exists(row):
    diary_no = str(row['diary_no'])
    for f in pdf_files:
        if f.startswith(diary_no):
            return True
    return False

# Apply function to each row
df['pdf_exists'] = df.apply(check_pdf_exists, axis=1)

# Summary
print("Judgements with matching PDFs:", df['pdf_exists'].sum())
print("Judgements without matching PDFs:", len(df) - df['pdf_exists'].sum())

Judgements with matching PDFs: 1937
Judgements without matching PDFs: 63


In [8]:
#Keep only rows with matching PDFs
df = df[df['pdf_exists']].reset_index(drop=True)

# Keep only relevant columns
columns_to_keep =  ['diary_no', 'case_no', 'judgment_dates', 'pdf_exists', 'pdf_path']
df = df[columns_to_keep]

# Save cleaned CSV
clean_csv_path = '/content/reduced_judgements_clean.csv'
df.to_csv(clean_csv_path, index=False)

print("Shape of cleaned dataset:", df.shape)
print("Columns after cleaning:", df.columns.tolist())
df.head()


Shape of cleaned dataset: (1937, 5)
Columns after cleaning: ['diary_no', 'case_no', 'judgment_dates', 'pdf_exists', 'pdf_path']


,diary_no,case_no,judgment_dates,pdf_exists,pdf_path
0,80384-1991,- 0,10-03-1992,True,/content/indian_judgments/pdfs/80384-1991___jo...
1,18732-1998,Crl.A. No.-001258-001263 - 1999,26-11-1999,True,/content/indian_judgments/pdfs/18732-1998___jo...
2,7011-2000,C.A. No.-002551-002551 - 2001,20-02-2008,True,/content/indian_judgments/pdfs/7011-2000___jon...
3,25859-2004,C.A. No.-000584-000584 - 2008,22-01-2008,True,/content/indian_judgments/pdfs/25859-2004___jo...
4,9232-2003,C.A. No.-004227-004227 - 2003,03-09-2009,True,/content/indian_judgments/pdfs/9232-2003___jon...


In [9]:
!pip install PyMuPDF
import fitz
from tqdm import tqdm

df['judgements_text'] = ''

# Filter to only non-empty PDFs
df = df[df['pdf_path'].apply(lambda x: os.path.exists(x) and os.path.getsize(x) > 0)].reset_index(drop=True)

for i, row in tqdm(df.iterrows(), total=len(df)):
    pdf_path = row['pdf_path']
    text = ''
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            page_text = page.get_text()
            if page_text:
                text += page_text + '\n'
        doc.close()
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    df.at[i, 'judgements_text'] = text

print("Text extraction done for", len(df), "PDFs")

100%|██████████| 1879/1879 [01:25<00:00, 22.10it/s]

Text extraction done for 1879 PDFs


In [10]:
df.head()[['diary_no', 'case_no', 'judgements_text']]


,diary_no,case_no,judgements_text
0,80384-1991,- 0,http://JUDIS.NIC.IN \nSUPREME COURT OF INDIA\n...
1,7011-2000,C.A. No.-002551-002551 - 2001,http://JUDIS.NIC.IN \nSUPREME COURT OF INDIA\n...
2,25859-2004,C.A. No.-000584-000584 - 2008,http://JUDIS.NIC.IN \nSUPREME COURT OF INDIA\n...
3,9232-2003,C.A. No.-004227-004227 - 2003,Page 1\nJUDGMENT\nREPORTABLE\nIN THE SUPREME C...
4,60150-1957,- 0,http://JUDIS.NIC.IN \nSUPREME COURT OF INDIA\n...


In [11]:
df.to_parquet('/content/judgements_with_text.parquet', index=False)

In [12]:
# Preview first 3 judgments fully
for i in range(1):
    print(f"--- Judgment {i+1}: {df['diary_no'][i]} ---\n")
    print(df['judgements_text'][i])  # full text, no slicing
    print("\n" + "-"*50 + "\n")


--- Judgment 1: 80384-1991 ---

http://JUDIS.NIC.IN 
SUPREME COURT OF INDIA
Page 1 of 26 
PETITIONER:
KUMAR PADMA PRASAD
        Vs.
RESPONDENT:
UNION OF INDIA AND ORS.
DATE OF JUDGMENT10/03/1992
BENCH:
KULDIP SINGH (J)
BENCH:
KULDIP SINGH (J)
SAWANT, P.B.
KASLIWAL, N.M. (J)
CITATION:
 1992 AIR 1213            1992 SCR  (2) 109
 1992 SCC  (2) 428        JT 1992 (2)   247
 1992 SCALE  (1)581
ACT:
     Constitution of India, 1950 :
     Articles  50, 136, 139A, 217(2), 233, 236(b),  237-High
Court   Judge-Appointment-Necessary   qualifications-Whether
Supreme  Court/High Court can exercise jurisdiction  on  the
ground that the incumbent does not fulfill qualifications as
required by Article 217.
     ‘Judicial Office’-Concept of-Explained.
     Government of Mizoram Notification CS/MZ/APPT/79  dated
23.6.1979-Assistant  to Deputy Commissioner appointed  under
Rules  8,9 and 15 of 1937-Rules for regulation of  procedure
of  officers to administer justice in Lushi Hills.  1937-Rs.
8,915-As

In [13]:
# STEP 0: Imports & Setup

import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models import LdaModel

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Initialize
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
CHUNK_WORDS = 400
OVERLAP = 50

# STEP 1: Preprocessing Function

def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # Remove line breaks, extra spaces
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()

    # Remove URLs, numbers, punctuation
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove capitalized words (heuristic proper nouns)
    tokens = text.split()
    tokens = [w for w in tokens if not w[0].isupper()]

    # Lowercase
    tokens = [w.lower() for w in tokens]

    # Remove stopwords, lemmatize, remove short words
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words and len(w) > 2]

    return ' '.join(tokens)

# Apply preprocessing
df['processed_text'] = df['judgements_text'].apply(preprocess_text)
# ===============================
# STEP 2: Chunking Function
# ===============================
def chunk_text_sliding_words(text, chunk_size=CHUNK_WORDS, overlap=OVERLAP):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return []

    words = text.split()
    if len(words) <= chunk_size:
        return [" ".join(words)]

    chunks = []
    start = 0
    while start < len(words):
        end = min(start + chunk_size, len(words))
        chunks.append(" ".join(words[start:end]))
        if end == len(words):
            break
        start += (chunk_size - overlap)
    return chunks

# Apply chunking
df['text_chunks'] = df['processed_text'].apply(chunk_text_sliding_words)
df_chunks = df.explode('text_chunks').reset_index(drop=True)
df_chunks = df_chunks[['diary_no', 'text_chunks']]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
#get the first two unique judgements
first_two_diary_nos = df_chunks['diary_no'].unique()[:2]

# Loop through each judgment and print all its chunks
for diary_no in first_two_diary_nos:
    print(f"\n=== Chunks for Judgment {diary_no} ===\n")

    # Get all chunks for this judgment
    chunks = df_chunks[df_chunks['diary_no'] == diary_no]['text_chunks'].tolist()

    # Print each chunk with its length in words
    for i, chunk in enumerate(chunks, 1):
        print(f"--- Chunk {i} ({len(chunk.split())} words) ---")
        print(chunk[:500] + ("..." if len(chunk) > 500 else ""))  # show first 500 chars
        print()


=== Chunks for Judgment 80384-1991 ===

--- Chunk 1 (400 words) ---
qualificationswhether exercise jurisdiction ground incumbent fulfill qualification required ofexplained dated appointed regulation procedure officer administer justice power analogous judicial hold judicial office judiciarynecessity justice assume jurisdiction judicial side probe matter purely administrative nature exclusively within purview serviceinterpretation letter dated addressed recommended name respondent appointment biodata enclosed said letter indicated respondent passing joined worked...

--- Chunk 2 (400 words) ---
one way criminal power conferred person hold held nay office officer holding various post executive often vested meet particular situation yet constitution framer provide source appointment high office amongst holder judicial office pfg personwho holding judicial office generic sense member judicial service eligible appointed person eligible appointed would mockery hold eligible appointed clear 

In [15]:
df.columns


Index(['diary_no', 'case_no', 'judgment_dates', 'pdf_exists', 'pdf_path',
       'judgements_text', 'processed_text', 'text_chunks'],
      dtype='object')

In [16]:
!pip install transformers sentence-transformers bertopic hdbscan umap-learn


In [18]:
!pip install transformers==4.30.0 sentence-transformers bertopic hdbscan umap-learn

  Using cached transformers-4.30.0-py3-none-any.whl.metadata (113 kB)
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached transformers-4.30.0-py3-none-any.whl (7.2 MB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


In [22]:
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import umap
import hdbscan
from bertopic import BERTopic


ImportError: cannot import name 'cached_download' from 'huggingface_hub' (/usr/local/lib/python3.12/dist-packages/huggingface_hub/__init__.py)

In [20]:
pip install --upgrade huggingface_hub


In [23]:
import re, time
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import umap
import matplotlib.pyplot as plt


In [28]:
CSV_PATH = "/content/judgements_with_text.parquet"   # adjust path if needed
TEXT_COLUMN = "processed_text"                  # column with extracted judgment text

df = pd.read_csv(CSV_PATH)
print("Available columns:", df.columns.tolist())
print("Total rows:", len(df))

# Filter non-empty texts
df = df[df[TEXT_COLUMN].notnull()].copy()
df = df.reset_index(drop=True)

# For quick run, take subset (increase later)
N_SAMPLES = 200
df = df.iloc[:N_SAMPLES]
print("Using", len(df), "documents")


UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 7-8: invalid continuation byte

In [ ]:
def preprocess_text(s):
    s = str(s).replace("\n", " ")
    s = re.sub(r"\s{2,}", " ", s)
    s = re.sub(r"(HON'BLE|JUDGE|CORAM).{0,50}", "", s, flags=re.IGNORECASE)
    return s.strip()

df["text_proc"] = df[TEXT_COLUMN].apply(preprocess_text)


In [ ]:
MODEL_NAME = "nlpaueb/legal-bert-base-uncased"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()


In [ ]:
MAX_TOKENS = 512
CHUNK_STRIDE = 50
BATCH_SIZE = 8

def chunk_text(text, max_len=MAX_TOKENS, stride=CHUNK_STRIDE):
    enc = tokenizer(text, return_tensors="pt", truncation=False, add_special_tokens=False)
    input_ids = enc["input_ids"][0].tolist()
    if len(input_ids) == 0:
        return []
    chunks = []
    start = 0
    while start < len(input_ids):
        end = start + max_len - 2
        chunk_ids = input_ids[start:end]
        chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True)
        chunks.append(chunk_text)
        if end >= len(input_ids):
            break
        start = max(0, end - stride)
    return chunks

def mean_pooling(token_embeddings, attention_mask):
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    summed = torch.sum(token_embeddings * mask, dim=1)
    counts = torch.clamp(mask.sum(dim=1), min=1e-9)
    return summed / counts

@torch.no_grad()
def document_to_embedding(text):
    chunks = chunk_text(text)
    if not chunks:
        return np.zeros(model.config.hidden_size, dtype=np.float32)
    embs = []
    for i in range(0, len(chunks), BATCH_SIZE):
        batch = chunks[i:i+BATCH_SIZE]
        enc = tokenizer(batch, padding=True, truncation=True, max_length=MAX_TOKENS, return_tensors="pt").to(device)
        out = model(**enc)
        pooled = mean_pooling(out.last_hidden_state, enc["attention_mask"])
        embs.append(pooled.cpu().numpy())
    doc_emb = np.mean(np.vstack(embs), axis=0)
    return doc_emb / (np.linalg.norm(doc_emb) + 1e-9)


In [ ]:
embeddings = []
for txt in tqdm(df["text_proc"].tolist(), desc="Embedding"):
    embeddings.append(document_to_embedding(txt))
embeddings = np.vstack(embeddings)
print("Embeddings shape:", embeddings.shape)


In [ ]:
N_CLUSTERS = 5
km = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
labels = km.fit_predict(embeddings)

df["cluster"] = labels

try:
    sil = silhouette_score(embeddings, labels)
    print("Silhouette Score:", sil)
except:
    print("Silhouette score skipped (too few samples)")


In [ ]:
cluster_to_category = {
    0: "Constitutional Law",
    1: "Administrative/Service Law",
    2: "Tax/Corporate Law",
    3: "Civil Law",
    4: "Criminal Law"
}

df["category"] = df["cluster"].map(cluster_to_category)


In [ ]:
OUTPUT_PATH = "judgments_with_categories.csv"
df[["diary_no", TEXT_COLUMN, "cluster", "category"]].to_csv(OUTPUT_PATH, index=False)
print("Saved:", OUTPUT_PATH)


In [ ]:
umap_emb = umap.UMAP(n_components=2, random_state=42).fit_transform(embeddings)
plt.figure(figsize=(8,6))
plt.scatter(umap_emb[:,0], umap_emb[:,1], c=labels, cmap="tab10", s=8)
plt.title("UMAP projection of judgments (5 clusters)")
plt.colorbar()
plt.show()
